# Pneumonia Detection Using Keras
This project aimed to create a three-way multi-classification model using a Convolutional Neural Network made with Keras to classify pneumonia from X-Rays. The purpose of this project is to create a quick and simplified way for anyone to classify pneumonia in children, possibly saving lives due to an early warning about the terrible disease.

### Trainer Specs
- Hardware
    - Intel i5-12600K
    - NVIDIA Geforce RTX 3060 Ti
        - Driver Version 552.22
        - CUDA Version 12.4
    - 32 GB Memory
- Software
    - Python 3.10.12
    - Tensorflow 2.16.1
    - Keras 3.1.1

### Background Information
This model is trained on [this dataset from Kaggle][kaggle_link]. However, Kaggle groups the bacterial and viral pneumonia together, creating a binary image classification problem. I wanted to take it a step further, separating the types of pneumonia out for further classification. To accomplish this, I renamed the `PNEUMONIA` directories to `BACTERIA` and moved all the viral pneumonia X-Rays into new `VIRUS` directories, to allow for easier data retrieval and labeling. To run locally, make sure to follow the same steps.

[kaggle_link]: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia "Chest X-Ray Images (Pneumonia)"

#### Setting up

In [1]:
from typing import Any
from keras.utils import PyDataset
from keras.models import Sequential
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from math import ceil
from skimage.io import imread
from skimage.transform import resize
from numpy import array, ndarray, eye
from os import listdir
from random import shuffle

HEIGHT = 512
WIDTH = 512

#### Custom Dataset Class
This custom class implements a Keras PyDataset, allowing for the model to read the data using a Python generator. This prevents massive memory usage and allows for higher definition images to be used. To represent the classes, I opted for one-hot encoding, meaning each image has an array attached to it representing the class, such as [1, 0, 0].

In [2]:
class PneumoniaDataset(PyDataset):
    def __init__(
        self: "PneumoniaDataset", dir: str, batch_size: int, **kwargs: Any
    ) -> None:
        super().__init__(**kwargs)
        self.x, self.y = self.get_image_paths(dir)
        self.batch_size = batch_size

    def __len__(self: "PneumoniaDataset") -> int:
        return ceil(len(self.x) / self.batch_size)

    def __getitem__(self: "PneumoniaDataset", idx: int) -> tuple[ndarray, ndarray]:
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]
        bad_output = [
            resize(imread(file_name), (HEIGHT, WIDTH)) for file_name in batch_x
        ]
        good_output = []
        for output in bad_output:
            new_output = (
                output.mean(axis=2) if output.shape == (HEIGHT, WIDTH, 3) else output
            )
            good_output.append(new_output)
        return array(good_output), batch_y

    def on_epoch_end(self: "PneumoniaDataset") -> None:
        indices = list(range(len(self.x)))
        shuffle(indices)
        self.x = [self.x[i] for i in indices]
        self.y = self.y[indices]

    def get_image_paths(self, dir: str) -> tuple[list[str], ndarray]:
        paths, classes = [], []
        class_labels = {"NORMAL": 0, "BACTERIA": 1, "VIRUS": 2}
        num_classes = len(class_labels)

        for label, class_idx in class_labels.items():
            label_paths = [
                f"{dir}/{label}/{file}" for file in listdir(f"{dir}/{label}")
            ]
            paths.extend(label_paths)
            classes.extend([class_idx] * len(label_paths))

        one_hot_labels = eye(num_classes)[classes]
        return paths, one_hot_labels

#### The Model
The model is a deep neural network using numerous convolution and pooling layers to extract information from each image. After each convolution layer, the outputs are normalized and regularized to prevent overfitting. Every convolution layer uses the ReLU activation, setting all negative values to 0, allowing for learning to be possible. As the problem for the model is image multi-classification, a softmax activation for the output and categorical crossentropy for the loss function are used.

In [3]:
model = Sequential(
    layers=[
        Input(shape=(HEIGHT, WIDTH, 1)),
        Conv2D(16, 3, padding="same", activation="relu"),
        BatchNormalization(),
        Conv2D(32, 3, padding="same", activation="relu"),
        Dropout(0.1),
        BatchNormalization(),
        MaxPooling2D(2),
        Conv2D(64, 3, padding="same", activation="relu"),
        BatchNormalization(),
        Conv2D(128, 3, padding="same", activation="relu"),
        Dropout(0.1),
        BatchNormalization(),
        MaxPooling2D(2),
        Conv2D(256, 3, padding="same", activation="relu"),
        BatchNormalization(),
        MaxPooling2D(2),
        Conv2D(512, 3, padding="same", activation="relu"),
        Dropout(0.1),
        BatchNormalization(),
        MaxPooling2D(2),
        Flatten(),
        Dense(3, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 512, 512, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512, 512, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 512, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64, 64, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 64, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │     1,572,867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,149,379 (12.01 MB)

 Trainable params: 3,147,363 (12.01 MB)

 Non-trainable params: 2,016 (7.88 KB)

#### Training
Now that everything is set up, it is time to train the model. This will be done with an instance of the PneumoniaDataset class, so as to not load every single image in the large dataset at once.

In [4]:
training_data = PneumoniaDataset("data/train", 4)

model.fit(x=training_data, epochs=64, verbose=2)

model.save("model0.keras")

Epoch 1/64


I0000 00:00:1715668802.115223 1280461 service.cc:145] XLA service 0x7ff71401b240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715668802.115318 1280461 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
I0000 00:00:1715668813.931890 1280461 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1434/1434 - 158s - 110ms/step - categorical_accuracy: 0.6703 - loss: 73.7212
Epoch 2/64
1434/1434 - 137s - 96ms/step - categorical_accuracy: 0.7063 - loss: 25.3073
Epoch 3/64
1434/1434 - 140s - 98ms/step - categorical_accuracy: 0.7307 - loss: 8.4815
Epoch 4/64
1434/1434 - 136s - 95ms/step - categorical_accuracy: 0.7643 - loss: 1.7321
Epoch 5/64
1434/1434 - 135s - 94ms/step - categorical_accuracy: 0.7994 - loss: 0.6015
Epoch 6/64
1434/1434 - 136s - 95ms/step - categorical_accuracy: 0.8127 - loss: 0.4625
Epoch 7/64
1434/1434 - 136s - 95ms/step - categorical_accuracy: 0.8059 - loss: 0.4651
Epoch 8/64
1434/1434 - 142s - 99ms/step - categorical_accuracy: 0.8085 - loss: 0.4743
Epoch 9/64
1434/1434 - 135s - 94ms/step - categorical_accuracy: 0.8106 - loss: 0.4780
Epoch 10/64
1434/1434 - 136s - 95ms/step - categorical_accuracy: 0.8114 - loss: 0.4575
Epoch 11/64
1434/1434 - 136s - 95ms/step - categorical_accuracy: 0.8282 - loss: 0.4670
Epoch 12/64
1434/1434 - 137s - 95ms/step - categorical_accur

#### Evaluation
Finally, the model's performance is ready for evaluation. This is done with a subset of the data that the model has never seen, so predictions must be made by the model.

In [9]:
test_data = PneumoniaDataset("data/test", 4)

model.evaluate(test_data)

31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - categorical_accuracy: 0.8450 - loss: 5.8666


[9.008214950561523, 0.707317054271698]

## Conclusions
And there we go! The model predicted the classifications for each image in the test data with 84% accuracy. To further improve this accuracy, different preprocessing techniques outside of simple resizing and transfer learning with a model such as ResNet could be employed. The uses of CNNs for medical diagnoses is an incredibly promising subject. With just one person doing some research and tinkering for while, a model with very good accuracy can be made. Altogether, I really enjoyed this project and look forward to implementing other applications of neural networks.